In [1]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import sys

In [2]:
#mouseのオルソログの情報を持ってくる。
def mouse_Orthologs():
    ortholog = browser.find_element(By.LINK_TEXT,"mouse")
    ortholog.click()
    
#Rattusのオルソログを持ってくる。
def Rattus_Ortholog():
    ortholog = browser.find_element(By.LINK_TEXT,"all")
    ortholog.click()
    #tag = browser.find_element_by_css_selector('a[data-ga-label="Rattus norvegicus"]')
    #ここ、コード書くときに詰まった。SeleniumでHTMLの出力→CSSセレクタでの指定が便利だ。
    tag = browser.find_element(By.CSS_SELECTOR,'a[data-ga-label="Rattus norvegicus"]')
    tag.click()
    #ここで新しいタブが開いてしまう。一旦新しいタブに操作を移動させたのち、昔のタブを消す。
    #こうするとタブはかさまず動き続けてくれる。
    sleep(5)
    browser.switch_to.window(browser.window_handles[1])
    browser.switch_to.window(browser.window_handles[0])
    browser.close()
    browser.switch_to.window(browser.window_handles[0])

#同じ方法で各動物のオルソログを取ってくる関数も書けるだろう。各動物種に対応可能。

In [3]:
browser= webdriver.Chrome()
df = pd.DataFrame()
gene_df = pd.DataFrame()

#Objective_genes.txtファイルに改行区切りで遺伝子名を入れておく。
with open('Objective_genes.txt') as f:
    for line in f.readlines():
        try:
            url= "https://www.ncbi.nlm.nih.gov/"
            browser.get(url)
            search_win = browser.find_element(By.ID,"term")
            search_win.send_keys(line)
            
            # searchボタンを押さなければいけない時と押さなくてもいい時がある(謎)これを解決
            try: 
                search_button = browser.find_element(By.ID,"search")
                search_button.click()
            except:
                pass
            
            #遺伝子名をAlso known asに指定される名前で検索してしまったとき、リンクのidが変わる問題を解決
            try: 
                gene = browser.find_element(By.ID,"feat_gene_title")
                gene.click()
            except:
                gene = browser.find_element(By.ID,"gene_title")
                gene.click()
            
            #humanの情報でいいときはここをコメントアウトする。ラットのオルソログがないものもあるので、その分エラーも増える。
            Rattus_Ortholog()
            #逆にマウスの情報が欲しい時は＃を消す
            #mouse_Orthologs()
            
            #gene一つずつ、辞書を作ってdfに入れては辞書を初期化する手法をとる。
            gene ={}
            name = browser.find_element(By.CLASS_NAME,"title")
            gene_url = browser.current_url
            
            gene["name"] = name.text
            gene["url"] = gene_url
            
            #Beautiful Soupで読み取るためのお膳立て(res)
            res = requests.get(gene_url)
            soup = BeautifulSoup(res.text,"html.parser")
            categoryItems = soup.find("dl",attrs={"id":"summaryDl"})
            categoryItems = categoryItems.find_all("dd")
            gene["gene_type"] = categoryItems[4].text
            gene["also_known_as"] = categoryItems[8].text
            gene["summary"] = categoryItems[9].text 
            #必要な情報が増える場合、ここで調節
            
            Location = soup.find("div",attrs={"class":"gt_cont_contents"})
            gene["Location"] = Location.find_all("span")[1].text
            #dfに個々の遺伝子のデータを順に追加していく。
            if "df" in globals():
                gene_df = pd.DataFrame(gene,index = [0])
                df = pd.concat([gene_df,df],axis=0)
            else:
                df= pd.DataFrame(gene,index = [0])
                
        #どこかしらでエラーが起きた場合はここへ。そのエラー内容をeで取得。
        except Exception as e:
            gene = {}
            gene["name"] = line
            #gene["error"] = e こちらだとエラー内容を出力可能だが、嵩む。
            gene["error"] = "error"
            gene_df = pd.DataFrame(gene,index = [0])
            df = pd.concat([gene_df,df],axis=0)
            
df.to_csv("gene.csv",index = False)
browser.quit()

In [4]:
gene_df
df
#joined

,name,url,gene_type,also_known_as,summary,Location,error
0,Irs1 insulin receptor substrate 1 [ Rattus nor...,https://www.ncbi.nlm.nih.gov/gene/25467/,protein coding,IRS1IRM,"Enables several functions, including SH2 domai...",9q34,NaN
0,Gck glucokinase [ Rattus norvegicus (Norway ra...,https://www.ncbi.nlm.nih.gov/gene/24385/,protein coding,GLK; GLUKA; RNGK2,"Enables several functions, including adenyl ri...",14q21,NaN
0,Gckr glucokinase regulator [ Rattus norvegicus...,https://www.ncbi.nlm.nih.gov/gene/25658/,protein coding,GLRE,"Enables several functions, including carbohydr...",6q14,NaN
0,Mtnr1b melatonin receptor 1B [ Rattus norvegic...,https://www.ncbi.nlm.nih.gov/gene/192646/,protein coding,Mt2; Mel1b-r,Predicted to enable G protein-coupled receptor...,8q12,NaN
0,G6pc2 glucose-6-phosphatase catalytic subunit ...,https://www.ncbi.nlm.nih.gov/gene/681817/,protein coding,Igrp; G6pc-rs,Predicted to enable glucose-6-phosphatase acti...,3q21,NaN
0,Notch2 notch receptor 2 [ Rattus norvegicus (N...,https://www.ncbi.nlm.nih.gov/gene/29492/,protein coding,Predicted to enable NF-kappaB binding activity...,"Biased expression in Spleen (RPKM 260.8), Lung...",2q34,NaN
0,Adamts9 ADAM metallopeptidase with thrombospon...,https://www.ncbi.nlm.nih.gov/gene/312566/,protein coding,Predicted to enable metalloendopeptidase activ...,Annotation category: suggests misassembly,4q34,NaN
0,THADA\n,NaN,NaN,NaN,NaN,NaN,Message: no such element: Unable to locate ele...
0,Tspan8 tetraspanin 8 [ Rattus norvegicus (Norw...,https://www.ncbi.nlm.nih.gov/gene/171048/,protein coding,CO-29; Tm4sf3,Enables integrin binding activity. Involved in...,7q22,NaN
0,Cdc123 cell division cycle 123 [ Rattus norveg...,https://www.ncbi.nlm.nih.gov/gene/116656/,protein coding,D123,Predicted to be involved in cell division. Pre...,17q12.3,NaN


<h1>Seleniumの強み</h1>

In [5]:
#url = "https://www.ncbi.nlm.nih.gov/gene/5468/ortholog/?scope=7776"

#res = requests.get(url)
#soup = BeautifulSoup(res.text,"html.parser")
#soup

#javascriptのせいで、オルソログのページのHTMLはBeautiful　Soupでは上手く取得できない。
#Seleniumから以下のよう(page_source)にする。すると人間が見たままのページのHTMLが出る。
#ここにSeleniumの強みがある。
#browser= webdriver.Chrome()
#browser.get(url)
#source_code = browser.page_source#これを参照しながらfind_elementをすれば良い。
#tag = browser.find_element_by_css_selector('a[data-ga-label="Rattus norvegicus"]')
#tag.click()
#categoryItems = soup.find("dl",attrs={"id":"summaryDl"})